In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer, classification_report, confusion_matrix, recall_score
from sklearn.feature_selection import RFE

import pickle

np.random.seed(13)

<img src="https://i.ibb.co/mGdcxkv/title.jpg">

# Classification of edible mushrooms
### Author: Georgi Stoyanov
#### October 2019

## Abstract
In this article you will see my methods for researching, manipulating, and preparing machine learning data, in which we try to build and train a model that qualifies mushrooms whether they are edible or poisonous. During the training process you will see how some of the data research assumptions turn out to be correct and the initial idea of testing several types of models fails due to the expectation of the same results as the __Logistic Regression__, and I start in another direction - __Feature Selection__. Also, once the final model was ready and its results also opened up, many interesting and intriguing ideas for further work opened up.

## Introduction
In the present article, we are going to analyze the mushroom dataset as made available by UCI Machine Learning<sup id="fnref:1"><a href="#fn:1" class="footnote">[1]</a></sup>. This tutorial is structured as follows. First, we are going to gain some domain knowledge on mushrooms. That will help in understanding the dataset features. Then we will run an exploratory analysis. Afterwards we will build models to classify mushrooms as edible or poisoned. References lists shown ahead about the overall article.

## Domain Knowledge<sup id="fnref:2"><a href="#fn:2" class="footnote">[2]</a></sup>

As anticipated, we are going to gain some basic domain knowledge about mushrooms.

### Mushrooms Basics Concepts
A mushroom, or toadstool, is the fleshy, spore-bearing fruiting body of a fungus, typically produced above ground on soil or on its food source.

The standard for the name “mushroom” is the cultivated white button mushroom, Agaricus bisporus; hence the word “mushroom” is most often applied to those fungi (Basidiomycota, Agaricomycetes) that have a stem (stipe), a cap (pileus), and gills (lamellae, sing. lamella) on the underside of the cap. “Mushroom” also describes a variety of other gilled fungi, with or without stems, therefore the term is used to describe the fleshy fruiting bodies of some Ascomycota. These gills produce microscopic spores that help the fungus spread across the ground or its occupant surface.

Forms deviating from the standard morphology usually have more specific names, such as “bolete”, “puffball”, “stinkhorn”, and “morel”, and gilled mushrooms themselves are often called “agarics” in reference to their similarity to Agaricus or their order Agaricales. By extension, the term “mushroom” can also designate the entire fungus when in culture; the thallus (called a mycelium) of species forming the fruiting bodies called mushrooms; or the species itself.

Identifying mushrooms requires a basic understanding of their macroscopic structure. Most are Basidiomycetes and gilled. Their spores, called basidiospores, are produced on the gills and fall in a fine rain of powder from under the caps as a result. At the microscopic level the basidiospores are shot off basidia and then fall between the gills in the dead air space. As a result, for most mushrooms, if the cap is cut off and placed gill-side-down overnight, a powdery impression reflecting the shape of the gills (or pores, or spines, etc.) is formed (when the fruit body is sporulating). The color of the powdery print, called a spore print, is used to help classify mushrooms and can help to identify them. Spore print colors include white (most common), brown, black, purple-brown, pink, yellow, and creamy, but almost never blue, green, or red.

Mushrooms are used extensively in cooking, in many cuisines (notably Chinese, Korean, European, and Japanese). Separating edible from poisonous species requires meticulous attention to detail; there is no single trait by which all toxic mushrooms can be identified, nor one by which all edible mushrooms can be identified. Many mushroom species produce secondary metabolites that can be toxic, mind-altering, antibiotic, antiviral, or bioluminescent. Although there are only a small number of deadly species, several others can cause particularly severe and unpleasant symptoms. Toxicity likely plays a role in protecting the function of the basidiocarp: the mycelium has expended considerable energy and protoplasmic material to develop a structure to efficiently distribute its spores<sup id="fnref:3"><a href="#fn:3" class="footnote">[3]</a></sup>.

### Mushroom Features Glossary
* Cap (Pileus): the expanded, upper part of the mushroom; whose surface is the pileus
* Cup (Volva): a cup-shaped structure at the base of the mushroom. The basal cup is the remnant of the button (the rounded, undeveloped mushroom before the fruiting body appears). Not all mushrooms have a cup.
* Gills (Lamellae): a series of radially arranged (from the center) flat surfaces located on the underside of the cap. Spores are made in the gills.
* Mycelial threads: root-like filaments that anchor the mushroom in the soli.
* Ring (Annulus): a skirt-like ring of tissue circling the stem of mature mushrooms. The ring is the remnant of the veil (the veil is the tissue that connects the stem and the cap before the gills are exposed and the fruiting body develops). Not all mushrooms have a ring.
* Scale: rough patches of tissue on the surface of the cap (scales are remnants of the veil).
* Stalk (or Stem, or Stape): the main support of the mushroom; it is topped by the cap. Not all mushrooms have a stalk (stem) <sup id="fnref:4"><a href="#fn:4" class="footnote">[4]</a></sup>.
Another feature to consider when identifying mushrooms is whether they bruise or bleed a specific color. Certain mushrooms will change colors when damaged or injured. Cutting into a mushroom and observing any color changes can be very important when trying to determine what it is <sup id="fnref:5"><a href="#fn:5" class="footnote">[5]</a></sup>.

A universal veil is a temporary membranous tissue that fully envelops immature fruiting bodies of certain gilled mushrooms. The developing Caesar’s mushroom (Amanita caesarea), for example, which may resemble a small white sphere at this point, is protected by this structure. The veil will eventually rupture and disintegrate by the force of the expanding and maturing mushroom, but will usually leave evidence of its former shape with remnants. These remnants include the volva, or cup-like structure at the base of the stipe, and patches or “warts” on top of the cap <sup id="fnref:6"><a href="#fn:6" class="footnote">[6]</a></sup>.

A partial veil (also called an inner veil, to differentiate it from the “outer” veil, or velum) is a temporary structure of tissue found on the fruiting bodies of some basidiomycete fungi, typically agarics. Its role is to isolate and protect the developing spore-producing surface, represented by gills or tubes, found on the lower surface of the cap. A partial veil, in contrast to a universal veil, extends from the stem surface to the cap edge. The partial veil later disintegrates, once the fruiting body has matured and the spores are ready for dispersal. It might then give rise to a stem ring, or fragments attached to the stem or cap edge. In some mushrooms, both a partial veil and a universal veil may be present <sup id="fnref:7"><a href="#fn:7" class="footnote">[7]</a></sup>.

### Mushroom Features by pictures
As shown by footnote <sup id="fnref:8"><a href="#fn:8" class="footnote">[8]</a></sup>, some pictures outline basic mushroom features as they can be found within our dataset.

#### Mushroom structure:
<img src="https://i.ibb.co/vQWDgNv/mushroom-glossary.jpg">

#### Mushroom cap shape:
<img src="https://i.ibb.co/DVPStCp/mushroom-cap-shape.jpg">

#### Mushroom cap surface:
<img src="https://i.ibb.co/StpLZWN/mushroom-cap-surface.jpg">

#### Mushroom gill attachment:
<img src="https://i.ibb.co/7vyFkvr/mushroom-gill-attachment.jpg">

#### Mushroom gill spacing:
<img src="https://i.ibb.co/Vjk3wGW/mushroom-gill-spacing.jpg">

#### Mushroom gill tissue arrangement:
<img src="https://i.ibb.co/YyKWrDx/mushroom-gill-tissue-arrangement.jpg">

#### Mushroom stalk type:
<img src="https://i.ibb.co/fvrhSvh/mushroom-stalk.jpg">

#### Mushroom ring type:
<img src="https://i.ibb.co/NyjyNfZ/mushroom-ring-type.jpg">

## Read dataset
Let's read the data and see if we read it correctly - size, structure, types, etc.

Parameter _na_\__values_ we use it for info from dataset:
_Missing Attribute Values: 2480 of them (denoted by "?"), All for attribute # 11._

In [ ]:
mushrooms_data = pd.read_csv("../input/mushroom-classification/mushrooms.csv", na_values = ["?"])
mushrooms_data.shape

In [ ]:
mushrooms_data.info()

In [ ]:
mushrooms_data.sample(10)

In [ ]:
mushrooms_data["class"].unique()

## Clean data
Once we have made sure everything is fine with the data loading, we can proceed to the next step.

Let's find the attributes that don't bring us information and remove them.

In [ ]:
mushrooms_data.apply(pd.Series.nunique)

We see that column _veil-type_ has the same value for all observations, which means that it does not give us any information. Let's remove it.

In [ ]:
mushrooms_data.drop("veil-type", axis = 1, inplace=True)
mushrooms_data.shape

Now we will check for missing data.

In [ ]:
mushrooms_data.isnull().sum()

Due to the specifics of our work - the classification of edible mushrooms, we can't replace, interpret or take any action on missing data. Therefore, we need to delete the entire attribute.

In [ ]:
mushrooms_data.drop("stalk-root", axis = 1, inplace=True)
mushrooms_data.shape

## Exploratory Analysis

The dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one <sup id="fnref:1"><a href="#fn:1" class="footnote">[1]</a></sup>.

According to dataset description, the first column represents the mushroom classification based on the two categories “edible” and “poisonous”. The other columns are:

1. cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
2. cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
3. cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
4. bruises: bruises=t, no=f
5. odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
6. gill-attachment: attached=a, descending=d, free=f, notched=n
7. gill-spacing: close=c, crowded=w, distant=d
8. gill-size: broad=b, narrow=n
9. gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
10. stalk-shape: enlarging=e, tapering=t
11. stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=? __Note: Removed in the previous step due to missing data__
12. stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
13. stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
14. stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
15. stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
16. veil-type: partial=p, universal=u __Note: Removed in the previous step due to insignificance__
17. veil-color: brown=n, orange=o, white=w, yellow=y
18. ring-number: none=n, one=o, two=t
19. ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
20. spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
21. population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
22. habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

For the next survey, we will create a function with __matplotlib__<sup id="fnref:9"><a href="#fn:9" class="footnote">[9]</a></sup> that will draw a classification bar graph of a given attribute. We do this to avoid code duplication, for better code readability, and for our convenience.

In [ ]:
def plot_attribute_class_bar(attribute):
    """Function to plot classification bar of given attribute"""
    plot_df = mushrooms_data[[attribute, "class"]]
    plot_df = plot_df.groupby([attribute, "class"]).size().unstack(fill_value=0)
    labels = plot_df.index
    edible_freq = plot_df.e
    poisonous_freq = plot_df.p
    
    x = np.arange(len(labels))  # the label locations
    width = 0.4  # the width of the bars
    
    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, edible_freq, width, label="Edible", color="g")
    rects2 = ax.bar(x + width/2, poisonous_freq, width, label="Poisonous", color="r")
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel("Frequency")
    ax.set_title("Classification bar of %s"%(attribute))
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    
    
    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate("{}".format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 0),
                        textcoords="offset points",
                        ha="center", va="bottom")
    
    
    autolabel(rects1)
    autolabel(rects2)
    
    fig.tight_layout()
    
    plt.show()

We are now ready to explore the attributes. Let's make bar graphs of all attributes and then describe our observations.

In [ ]:
for column in mushrooms_data.columns[1:]:
    plot_attribute_class_bar(column)

Main insights resulting from above barplots are:

* only poisonous mushrooms have convex cap-shape; only edible mushrooms have sunken cap-shape
* only poisonous mushrooms have cap-surface with grooves
* only edible mushrooms have green or purple cap-color
* odor is strongly indicative of what mushrooms are (edible/poisonous)
* only poisonous mushrooms have buff or green gill color; only edible mushrooms have red or orange gill color
* stalk_color_above_ring and stalk_color_below_ring are relevant features for out classification problem
* only edible mushrooms have brown or orange veil color; only poisonous mushrooms have yellow veil color
* only poisonous mushrooms do not have rings
* only edible mushrooms have flaring ring type; only poisonous mushrooms have none or large ring type
* only edible mushrooms have black, orange, purple or yellow spore print color; only poisonous mushrooms have green spore print color
* only edible mushrooms have abundant or numerous population
* only edible musrooms have waste type habitat

As many of the attributes we see describe classes very well, this predisposes to a high predictive coefficient for a classification models. Let's check the assumption.

## Machine Learning with Classification Methods
Before we start training, we need to analyze and process data a little more.

### Analyze and process data
We will now examine what sample of data there is for each class, and how balanced they are.

In [ ]:
mushrooms_data.groupby("class").size() / len(mushrooms_data)

Data seems to have a perfectly balanced sample of both classes.

Let's separate the class column so we get two sets of attributes and labels.

In [ ]:
mushrooms_data_attributes = mushrooms_data.drop("class", axis = 1)
mushrooms_data_attributes.shape

In [ ]:
mushrooms_data_labels = mushrooms_data["class"]
mushrooms_data_labels.shape

Now we have to expand the categories, operation unmelt, this is easily achieved with the pandas function _get_\__dummies_.

In [ ]:
mushrooms_data_attributes = pd.get_dummies(mushrooms_data_attributes)
mushrooms_data_attributes.shape

In [ ]:
mushrooms_data_attributes.head(10)

From the original 20 columns we now have 111 :)
All columns contain only 1 or 0, so we do not need to scale the data.

We will divide the sets into 3 parts. One part will be for training, the second for validation and the third we will use finally, after selecting the best model, for testing. The ratio will be 60%, 15% and 25%.

In [ ]:
attributes_train, attributes_test, labels_train, labels_test = train_test_split(
    mushrooms_data_attributes, mushrooms_data_labels, train_size = 0.75, stratify = mushrooms_data_labels)

In [ ]:
attributes_train, attributes_val, labels_train, labels_val = train_test_split(
    attributes_train, labels_train, train_size = 0.80, stratify = labels_train)

In [ ]:
attributes_train.shape, attributes_val.shape, attributes_test.shape

In [ ]:
labels_train.shape, labels_val.shape, labels_test.shape

We are finally ready for machine learning. Let's go!

### Logistic Regression
We start with the default settings.

In [ ]:
mushroom_model = LogisticRegression(solver = "lbfgs")
mushroom_model.fit(attributes_train, labels_train)

In [ ]:
mushroom_model.score(attributes_train, labels_train)

In [ ]:
mushroom_model.score(attributes_val, labels_val)

Hmm interesting, obviously our model overfit data.
Well, let's see if that is the case. We will use __Grid Search Cross Validation__ with __Stratified KFold__ to split the data into 10 smaller test sets, which we do for greater diversity in training data.
#### Grid Search Cross Validation with Stratified KFold

In [ ]:
params = {
    "C": [1e-7, 1e-5, 1e-4, 1e-3, 0.01, 0.1],
    "fit_intercept": [True, False]
}
skf = StratifiedKFold(n_splits=10, random_state=None)
k_fold = skf.split(attributes_train, labels_train)
grid_search = GridSearchCV(LogisticRegression(solver="lbfgs"), params, make_scorer(f1_score, pos_label = "e"), cv = k_fold)
grid_search.fit(attributes_train, labels_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_estimator = grid_search.best_estimator_
best_estimator.fit(attributes_train, labels_train)

In [ ]:
best_estimator.score(attributes_train, labels_train)

In [ ]:
best_estimator.score(attributes_val, labels_val)

In [ ]:
grid_search.cv_results_

In [ ]:
print(classification_report(labels_val, best_estimator.predict(attributes_val)))

From the results obtained, the fact that the differences in the results between the folds are negligibly small, we can conclude that our model is probably not overfitting. Attributes may very well describe both classes. Let's change the approach and try to make a __Feature Selection__<sup id="fnref:10"><a href="#fn:10" class="footnote">[10]</a></sup>. For this we will use __RFE (Recursive Feature Elimination)__<sup id="fnref:10"><a href="#fn:10" class="footnote">[10]</a></sup>.
#### Feature Selection with RFE (Recursive Feature Elimination)<sup id="fnref:10"><a href="#fn:10" class="footnote">[10]</a></sup>
The RFE method works by recursively removing attributes and building a model on those attributes that remain. It uses accuracy metric to rank the feature according to their importance. The RFE method takes the model to be used and the number of required features as input. It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.

In [ ]:
model = LogisticRegression(solver="lbfgs")
rfe = RFE(model, 6)
X_rfe = rfe.fit_transform(attributes_train,labels_train)
model.fit(X_rfe,labels_train)
print(rfe.support_)
print(rfe.ranking_)

Here we took LinearRegression model with 6 features and RFE gave feature ranking as above, but the selection of number ‘6’ was random. Now we need to find the optimum number of features, for which the accuracy is the highest. We do that by using loop starting with 1 feature and going up to 20. We then take the one for which the accuracy is highest.

In [ ]:
#no of features
nof_list=np.arange(1,20)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LogisticRegression(C = 1e9, solver="lbfgs")
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(attributes_train,labels_train)
    X_test_rfe = rfe.transform(attributes_val)
    model.fit(X_train_rfe,labels_train)
    score = model.score(X_test_rfe,labels_val)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

As seen from above code, the optimum number of features is 7. We now feed 7 as number of features to RFE and get the final set of features given by RFE method, as follows:

In [ ]:
cols = list(attributes_train.columns)
model = LogisticRegression(solver="lbfgs")
#Initializing RFE model
rfe = RFE(model, nof)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(attributes_train,labels_train)  
#Fitting the data to model
model.fit(X_rfe,labels_train)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Now that we have sets with optimal column names we need to filter the 3 attribute sets.

In [ ]:
opt_attributes_train = attributes_train[selected_features_rfe]
opt_attributes_val = attributes_val[selected_features_rfe]
opt_attributes_test = attributes_test[selected_features_rfe]

In [ ]:
opt_attributes_train.shape, opt_attributes_val.shape, opt_attributes_test.shape

We are now ready to train the final model.
### Logistic Regression with future selected data

In [ ]:
opt_mushroom_model = LogisticRegression(solver = "lbfgs")
opt_mushroom_model.fit(opt_attributes_train, labels_train)

In [ ]:
opt_mushroom_model.score(opt_attributes_train, labels_train)

In [ ]:
opt_mushroom_model.score(opt_attributes_val, labels_val)

In [ ]:
opt_mushroom_model.score(opt_attributes_test, labels_test)

In [ ]:
print(classification_report(labels_test, opt_mushroom_model.predict(opt_attributes_test)))

In [ ]:
confusion_matrix(labels_test, opt_mushroom_model.predict(opt_attributes_test))

In [ ]:
recall_score(labels_test, opt_mushroom_model.predict(opt_attributes_test), pos_label = "p")

In [ ]:
recall_score(labels_test, opt_mushroom_model.predict(opt_attributes_test), pos_label = "e")

Finally, let's save the trained model so that we can load and use it later without having to train again. We will use __pickle__<sup id="fnref:11"><a href="#fn:11" class="footnote">[11]</a></sup>.

In [ ]:
## Save the model to disk
#filename = "../input/models/final_model.p"
## Store data (serialize)
#with open(filename, "wb") as handle:
#    pickle.dump(opt_mushroom_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
# 
## Some time later...
# 
## Load the model from disk
## Load data (deserialize)
#with open(filename, "rb") as handle:
#    loaded_model = pickle.load(handle)
#
#assert(loaded_model.score(opt_attributes_test, labels_test) == opt_mushroom_model.score(opt_attributes_test, labels_test))

## Conclusion

In conclusion, we can say that we have achieved more than good results. The datаset turned out to be well balanced from to sample for both classes. We also found that attributes very well describe and differentiate classes. We even managed to reduce the attributes from 111 to 7 without losing precision. In my opinion, any qualification method wouldn't have problems with this dataset. Of course, this opens up many ways to experiment with data and different models. For example, if you noticed with the selected model, __we rate 10 poisonous mushrooms as edible__!?!? __This is dangerous for this type of qualifications, especially with the actual use of the model__! But come on, more ideas in the next section :)

## Future work

The following things can be done as future actions:
* Removing the current minimal error, especially when determining poisonous species. This can be done by adding more attributes or using different models - trees, forests, etc.
* It would be very interesting to remove attribute groups for a specific type of qualification, such as:
  1. Mushrooms detached and away from their habitat: In this case, we cannot know some of the attributes (stalk, population,        habitat ...), so we have to exclude them and train a new model using the same method described in the article.
  2. As we know, odor is very relatively measurable, except for chemical analysis or other similar methods. For this we can          eliminate these attributes.
  3. For colors, people may also define them differently. We are removing these color-related attributes.
  4. Combinations, inversions and any other kind of interpretation of the ideas described here.
* Trying to find other data and process it with trained models.

## Refferences

<div class="footnotes">
  <ol>
      <li id="fn:1">
       <p><a href="https://archive.ics.uci.edu/ml/datasets/mushroom" rel="noopener" target="_blank"> UCI Machine Learning Archive – Mushroom Dataset</a><a href="#fnref:1" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
    <li id="fn:2">
       <p><a href="https://datascienceplus.com/mushrooms-classification-part-1/">Mushrooms Classification – Part 1</a> by <a href="https://datascienceplus.com/author/giorgiogarziano/">Giorgio Garziano</a><a href="#fnref:2" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:3">
       <p><a href="https://en.wikipedia.org/wiki/Mushroom" rel="noopener" target="_blank">Wikipedia – Mushroom Tutorial</a><a href="#fnref:3" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:4">
       <p><a href="http://www.enchantedlearning.com/subjects/fungi/label/mushroom/" rel="noopener" target="_blank">Mushroom Anatomy</a><a href="#fnref:4" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:5">
       <p><a href="https://www.mushroom-appreciation.com/identifying-mushrooms.html" rel="noopener" target="_blank">Identify Mushrooms</a><a href="#fnref:5" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:6">
       <p><a href="https://en.wikipedia.org/wiki/Universal_veil" rel="noopener" target="_blank">Wikipedia – Universal Veil</a><a href="#fnref:6" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:7">
       <p><a href="https://en.wikipedia.org/wiki/Partial_veil" rel="noopener" target="_blank">Wikipedia – Partial Veil</a><a href="#fnref:7" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:8">
       <p><a href="https://www.usask.ca/biology/fungi/glossary.html" rel="noopener" target="_blank">Mushroom Glossary</a><a href="#fnref:8" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:9">
       <p><a href="https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html" rel="noopener" target="_blank">Grouped bar chart with labels</a><a href="#fnref:9" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:10">
       <p><a href="https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b" rel="noopener" target="_blank">Feature Selection with sklearn and Pandas</a> by <a href="https://towardsdatascience.com/@abhini.shetye" rel="noopener" target="_blank">Abhini Shetye</a><a href="#fnref:10" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
      <li id="fn:11">
       <p><a href="https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict" rel="noopener" target="_blank">How can I use pickle to save a dict?</a> answer by <a href="https://stackoverflow.com/users/464744/blender" rel="noopener" target="_blank">Blender</a><a href="#fnref:11" class="reversefootnote">&#8617;</a></p>
    </li>
      <br/>
  </ol>
    <p><a href="https://towardsdatascience.com/building-a-perfect-mushroom-classifier-ceb9d99ae87e" rel="noopener" target="_blank">Building a Perfect Mushroom Classifier</a> by <a href="https://towardsdatascience.com/@marcopeixeiro" rel="noopener" target="_blank">Marco Peixeiro</a></p>
</div>
    <p><a href="https://www.kaggle.com/turksoyomer/classification-methods-on-mushroom-dataset" rel="noopener" target="_blank">Classification Methods on Mushroom Dataset</a> by <a href="https://www.kaggle.com/turksoyomer" rel="noopener" target="_blank">Omer Turksoy</a></p>
</div>